In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae_tmp1', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.05, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

In [17]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    means_topic_summary = tf.reduce_mean(means_topic_infer, 0)
    
    summary_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic_summary)
    summary_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(summary_dec_initial_state, multiplier=config.beam_width)
    summary_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic_summary, multiplier=config.beam_width) # added
    
    summary_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=summary_beam_dec_initial_state,
        beam_width=config.beam_width,
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=summary_beam_latents_input)

    summary_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        summary_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    summary_beam_output_token_idxs = summary_beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [18]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [19]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [20]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [21]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [22]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [23]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [24]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'TRUE: %s' % true_sent)
        print(i, 'PRED: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_topic_sent)
        
def print_summary(test_batch):
    feed_dict = get_feed_dict(test_batch)
    feed_dict[t_variables['batch_l']] = config.n_topic
    feed_dict[t_variables['keep_prob']] = 1.
    pred_topics_freq_bow_indices, pred_summary_token_idxs = sess.run([topics_freq_bow_indices, summary_beam_output_token_idxs], feed_dict=feed_dict)
    pred_summary_sents = idxs_to_sents(pred_summary_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Output sentences for each topic-----------')
    print('Item idx:', test_batch[0].item_idx)
    for i, (topic_freq_bow_idxs, pred_summary_sent) in enumerate(zip(topics_freq_bow_idxs, pred_summary_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_summary_sent)
        
    print('-----------Summaries-----------')
    for i, summary in enumerate(test_batch[0].summaries):
        print('SUMMARY %i :'%i, '\n', summary)

In [25]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','LM','','VALID:','TM','','','','LM','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','NLL','KL','LOSS','PPL','NLL','KL','REG','NLL','KL', 'Beta']]))))

In [26]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, sent_loss_recon_dev, sent_loss_kl_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
            if loss_dev < loss_min:
                loss_min = loss_dev
                saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, '%.2f'%sent_loss_recon_train, '%.2f'%sent_loss_kl_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, '%.2f'%sent_loss_recon_dev, '%.2f'%sent_loss_kl_dev,  '%.3f'%beta_eval],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            print_summary(test_batches[1][1])
            print_sample(batch)
            
            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                        LM        VALID:  \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG   NLL    KL    LOSS   
1        14   0     0  123.45  1035  113.05  0.37  0.90  9.13  1.46  126.48   
501      62   0   500  121.19   598  113.84  0.17  0.63  6.50  1.38  111.52   
1001     71   0  1000  120.10   568  113.13  0.26  0.49  6.15  1.07  111.01   
1501     71   0  1500  119.68   553  112.78  0.39  0.40  6.03  0.91  110.78   
2001     63   0  2000  119.22   542  112.33  0.49  0.33  5.96  0.80  110.57   
2276     39   1     0  118.98   536  112.10  0.54  0.31  5.93  0.75  110.24   
2776     52   1   500  118.70   529  111.80  0.61  0.27  5.90  0.68  110.28   
3276     63   1  1000  118.41   523  111.50  0.67  0.24  5.87  0.63  110.10   
3776     62   1  1500  118.35   517  111.42  0.72  0.22  5.85  0.58  109.95   
4276     62   1  2000  118.21   512  111.27  0.77  0.20  5.83  0.55  109.83   
4551     40   2     0  118.16   510  111.21  0.79  0.20  5.81  0.53  109.75   
5051     62   2   500  118.02   506  111.07  0.82  0.18  5.79  0.50  109.62   
5551     64   2  1000  117.88   503  110.93  0.86  0.17  5.76  0.48  109.61   
6051     64   2  1500  117.80   500  110.85  0.89  0.16  5.74  0.46  109.52   
6551     64   2  2000  117.68   497  110.74  0.91  0.15  5.71  0.45  109.37   
6826     41   3     0  117.66   496  110.72  0.93  0.14  5.69  0.44  109.18   
7326     64   3   500  117.57   493  110.63  0.95  0.14  5.66  0.43  109.07   
7826     55   3  1000  117.49   491  110.56  0.97  0.13  5.64  0.42  109.12   
8326     53   3  1500  117.43   489  110.51  0.99  0.12  5.61  0.41  109.18   
8826     52   3  2000  117.37   487  110.45  1.01  0.12  5.58  0.40  109.08   
9101     39   4     0  117.31   487  110.40  1.02  0.11  5.57  0.40  108.90   
9601     53   4   500  117.23   485  110.33  1.04  0.11  5.55  0.39  109.11   
10101    52   4  1000  117.13   483  110.24  1.06  0.10  5.52  0.38  109.01   
10601    71   4  1500  117.12   482  110.23  1.08  0.10  5.50  0.38  108.85   
11101    53   4  2000  117.05   480  110.16  1.09  0.10  5.48  0.37  108.91   
11376    39   5     0  117.04   480  110.16  1.10  0.09  5.47  0.37  108.64   
11876    62   5   500  116.97   478  110.11  1.12  0.09  5.45  0.37  108.46   
12376    65   5  1000  116.91   477  110.06  1.13  0.09  5.43  0.37  108.39   
12876    55   5  1500  116.84   476  110.00  1.15  0.09  5.41  0.37  108.52   
13376    55   5  2000  116.81   475  109.98  1.16  0.08  5.40  0.37  108.51   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...   ...     ...   
100101   30  44     0  114.67   421  107.96  1.91  0.02  4.49  0.40  107.05   
100601   54  44   500  114.66   421  107.95  1.91  0.02  4.48  0.40  106.98   
101101   54  44  1000  114.66   421  107.95  1.92  0.02  4.48  0.40  107.11   
101601   59  44  1500  114.66   421  107.95  1.92  0.02  4.48  0.40  107.08   
102101   58  44  2000  114.65   421  107.94  1.92  0.02  4.48  0.40  106.94   
102376   33  45     0  114.65   420  107.94  1.92  0.02  4.48  0.40  106.67   
102876   69  45   500  114.64   420  107.94  1.92  0.02  4.48  0.40  106.61   
103376   69  45  1000  114.64   420  107.94  1.92  0.02  4.47  0.40  106.59   
103876   61  45  1500  114.63   420  107.93  1.92  0.02  4.47  0.40  106.72   
104376   61  45  2000  114.63   420  107.93  1.92  0.02  4.47  0.40  106.61   
104651   34  46     0  114.63   420  107.93  1.93  0.02  4.47  0.40  106.71   
105151   61  46   500  114.62   420  107.92  1.93  0.02  4.47  0.40  106.69   
105651   61  46  1000  114.62   420  107.92  1.93  0.02  4.46  0.40  106.92   
106151   63  46  1500  114.61   420  107.91  1.93  0.02  4.46  0.40  106.95   
106651   62  46  2000  114.61   420  107.91  1.93  0.02  4.46  0.40  106.81   
106926   34  47     0  114.61   420  107.91  1.93  0.02  4.46  0.40  106.91   
107426   58  47   500  114.61   420  107.91  1.93  0.02  4.46  0.40  107.06   
107926   57  47  1000  114.60   419  107.90  1.93  0.02  4.45  0.40  106.

-----------Output sentences for each topic-----------
Item idx: B000VB7EFW
0  BOW: bottom top plastic piece corners part edges speck cracked feet
0  SENTENCE: it 's a great case for the price
1  BOW: quality price made 'm nice ... time - thing put
1  SENTENCE: i would recommend this to anyone looking for a laptop case
2  BOW: cover color keyboard mac pro hard apple easy nice perfectly
2  SENTENCE: the keyboard cover fits perfectly and is easy to put on
3  BOW: ! love recommend perfect color perfectly highly bought ordered happy
3  SENTENCE: i love the color and the color is great ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
4  BOW: sleeve & ; protection air pro protect inside inch snug
4  SENTENCE: the sleeve is a little snug , but it does not fit my macbook pro # & # # ;
5  BOW: zipper strap side - : padding zippers top compartment velcro
5  SENTENCE: it is a little bigger than i expected
6  BOW: carry pockets back compartments comfortable shoulder room stuff space wo

# confirm variables

In [27]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [28]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([0.02120299, 0.32619268, 0.02783219, 0.08510477, 0.04270138,
        0.05854441, 0.17855252, 0.17987773, 0.05455504, 0.02543635],
       dtype=float32),
 array([0.02938788, 0.24803634, 0.00798052, 0.01799672, 0.03427965,
        0.15884441, 0.19315946, 0.1489072 , 0.1465453 , 0.01486251],
       dtype=float32))

In [29]:
_means_topic_infer[0][:, :4]

array([[-0.3878181 , -0.52203584, -1.9560823 ,  0.43151194],
       [ 0.22494435, -0.10070819, -0.9306398 ,  0.24482146],
       [-1.7151399 , -1.2774523 , -4.6163416 ,  0.35484877],
       [-0.5554816 ,  0.38961172, -3.5819693 , -2.0564005 ],
       [-0.6458729 , -1.167951  , -1.8318795 ,  1.5787079 ],
       [ 0.67180693, -0.01855016,  0.19753097,  0.6514791 ],
       [ 1.7514257 ,  0.7512727 ,  2.0913913 ,  0.32178992],
       [ 0.05339959, -0.08493266, -1.3765373 , -0.00502022],
       [ 0.65437484,  0.2057987 , -0.09499581,  0.10592981],
       [ 1.4489527 ,  0.29247132,  1.894986  ,  0.97273844]],
      dtype=float32)

In [30]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [31]:
for idxs in pred_topics_freq_bow_idxs:
    print([idx_to_word[idx] for idx in idxs])

['bottom', 'top', 'plastic', 'part', 'piece', 'speck', 'feet', 'corners', 'edges', 'rubber']
['quality', 'price', 'made', "'m", 'nice', '...', 'time', '-', 'thing', 'lot']
['cover', 'color', 'keyboard', 'mac', 'pro', 'apple', 'hard', 'easy', 'perfectly', 'air']
['!', 'love', 'color', 'recommend', 'perfect', 'perfectly', 'bought', 'highly', 'buy', 'purchase']
['sleeve', '&', ';', 'protection', 'pro', 'air', 'inside', 'protect', 'inch', 'snug']
['zipper', 'strap', '-', 'side', ':', 'zippers', 'padding', 'top', 'velcro', 'compartment']
['carry', 'pockets', 'back', 'compartments', 'space', 'room', 'comfortable', 'work', 'shoulder', 'stuff']
['bought', 'time', '$', 'return', 'money', 'back', 'item', 'buy', 'received', '...']
['handle', 'years', 'months', 'straps', 'broke', 'strap', 'wheels', 'started', 'zippers', 'year']
['pocket', 'room', 'ipad', 'power', 'mouse', 'small', 'charger', 'perfect', 'carry', 'netbook']


In [32]:
pred_topic_embeddings[:, :10]

array([[-1.67144597e-01,  4.32745188e-01, -1.22095034e-01,
        -1.03223495e-01,  1.78706244e-01, -1.67346388e-01,
        -1.68974623e-01,  1.05840132e-01,  2.28138641e-01,
        -4.37488645e-01],
       [ 2.97464505e-02,  1.45410940e-01, -1.46664038e-01,
         3.29389051e-02,  8.52175057e-02,  3.31251651e-01,
         4.63549852e-01,  1.16298549e-01,  1.64733216e-01,
         2.66190320e-01],
       [-4.47264075e-01,  6.11892581e-01, -2.60540456e-01,
         2.68532097e-01,  5.53756813e-03, -7.70700499e-02,
         2.44523048e-01, -2.98057441e-02,  4.49646443e-01,
         5.72897978e-02],
       [ 4.71091345e-02, -2.23770499e-01, -3.51872534e-01,
         3.92161198e-02, -2.27904618e-01, -8.74633193e-02,
         4.43780005e-01,  1.71568878e-02, -2.76746869e-01,
         4.64234948e-01],
       [ 1.28112612e-02, -3.28971356e-01, -2.27453679e-01,
         2.29710922e-01,  4.15110923e-02, -9.82860252e-02,
        -2.48736888e-02, -5.07779300e-01, -3.58882606e-01,
        -8.

In [33]:
pred_topic_bow

array([[9.63746061e-07, 1.27285850e-04, 1.01267084e-04, ...,
        2.11540428e-05, 5.35642539e-05, 1.34317424e-05],
       [2.02581621e-04, 5.98845724e-03, 6.64945692e-04, ...,
        1.31005127e-05, 6.20674400e-04, 3.25565743e-05],
       [3.54659511e-04, 1.93204498e-04, 1.91751606e-04, ...,
        4.05486617e-06, 1.38744406e-06, 2.71618569e-06],
       ...,
       [2.07204619e-04, 2.19707973e-02, 1.25209999e-03, ...,
        9.57436669e-07, 3.18203602e-05, 2.10850635e-06],
       [3.60615218e-06, 1.41350058e-04, 1.86732781e-04, ...,
        3.07424016e-05, 2.13968139e-02, 5.15784377e-05],
       [9.11835741e-05, 2.65167218e-05, 1.20110635e-05, ...,
        4.97434940e-03, 6.13421784e-04, 8.76899168e-04]], dtype=float32)

In [34]:
_w_means_topic

array([[ 0.002856  , -0.12888564, -0.02525959, ...,  0.13936642,
         0.06750863,  0.05765012],
       [-0.04506291, -0.0032232 , -0.23664857, ...,  0.13413484,
         0.00195352,  0.04439881],
       [-0.20138465, -0.21933417, -0.45687827, ...,  0.27684236,
        -0.0871457 ,  0.12629426],
       ...,
       [-0.1734638 ,  0.02492818, -0.16350251, ...,  0.15120973,
         0.06725367,  0.02727391],
       [ 0.21558905,  0.00539135,  0.21533233, ..., -0.0434513 ,
         0.09895361,  0.04902754],
       [ 0.21534637,  0.07677771,  0.08166326, ..., -0.14705855,
        -0.02142379, -0.00574149]], dtype=float32)

In [35]:
_b_means_topic

array([ 1.7616607e-01,  3.6224346e-03, -3.0754781e-01,  1.3990225e-01,
        3.1391117e-01, -1.0993136e-01,  1.7876926e-01,  2.0730966e-01,
        1.5003067e-01, -3.9625127e-02,  2.5470558e-01, -5.3844504e-02,
        2.3800465e-01,  3.0333728e-01, -5.1428996e-02, -2.0459709e-01,
        8.4517933e-02,  2.4236085e-01,  1.1645479e-01, -1.2791230e-01,
        2.7641625e-05, -9.3722373e-02,  1.5119699e-01, -9.8118551e-02,
        6.1052620e-02, -3.0996993e-01,  9.3275547e-02, -2.3359668e-02,
        3.0665058e-01,  3.2136410e-02,  3.5929924e-01,  1.1415872e-01],
      dtype=float32)

In [36]:
pred_means_topic

array([[-3.94786239e-01, -5.32271326e-01, -1.96238506e+00,
         4.46755648e-01,  1.01060510e+00, -1.22109279e-01,
         1.50481164e-02,  2.69290149e-01,  7.44255066e-01,
        -8.88179123e-01,  4.17418182e-01,  3.42713386e-01,
         7.09146023e-01,  1.00336218e+00, -2.45433211e-01,
         7.27762282e-02, -1.27207696e-01,  9.71992254e-01,
         5.58246493e-01, -3.56951237e-01,  2.25926831e-01,
        -7.56059289e-01,  9.44043159e-01,  3.91685903e-01,
        -1.83836803e-01, -6.32101893e-01,  6.59335405e-03,
         2.12992758e-01,  1.09152269e+00,  7.77865648e-01,
         5.01754999e-01,  2.74188370e-01],
       [ 2.16301292e-01, -1.04559906e-01, -9.37525332e-01,
         2.46304855e-01,  8.65785837e-01, -2.16787368e-01,
         2.97387868e-01,  4.00805056e-01,  4.88129437e-01,
        -2.57460028e-01,  4.59877938e-01,  4.12266478e-02,
         5.00950575e-01,  7.02583432e-01, -4.87488061e-02,
        -3.55852962e-01,  9.70679596e-02,  6.36684895e-01,
         3.26

In [37]:
_w_mean_topic_infer[:, :10, 0]

array([[-3.98322241e-03,  1.24624174e-03, -8.86734575e-03,
        -1.33664701e-02,  2.19804631e-03,  1.40512912e-02,
        -2.01842021e-02, -1.48272878e-02, -1.00363810e-02,
         4.16810438e-03],
       [-2.98862299e-03,  1.12401508e-03,  1.98944081e-02,
         8.91935173e-03, -1.72090791e-02,  1.18043087e-02,
         5.85581455e-03, -1.30022550e-03,  1.13905175e-02,
         7.59256352e-03],
       [-2.46382388e-03, -1.32139903e-02,  2.27825576e-03,
        -2.34090518e-02,  1.94362942e-02, -5.95725654e-03,
         1.02149220e-02,  1.48489103e-02, -4.36442345e-03,
        -4.79104137e-03],
       [-1.12793921e-02, -1.19534284e-02, -1.76024586e-02,
         1.57898106e-02, -2.07192414e-02,  1.61588797e-03,
         2.66234428e-02,  2.25266698e-03,  2.02661511e-02,
        -1.05920928e-02],
       [-4.38706949e-03, -4.19018278e-03,  2.24646833e-03,
         5.32911252e-03,  1.03885671e-02,  7.60490214e-03,
         1.11821210e-02,  1.38774859e-02, -7.36943027e-03,
        -7.

In [38]:
_b_means_topic

array([ 1.7616607e-01,  3.6224346e-03, -3.0754781e-01,  1.3990225e-01,
        3.1391117e-01, -1.0993136e-01,  1.7876926e-01,  2.0730966e-01,
        1.5003067e-01, -3.9625127e-02,  2.5470558e-01, -5.3844504e-02,
        2.3800465e-01,  3.0333728e-01, -5.1428996e-02, -2.0459709e-01,
        8.4517933e-02,  2.4236085e-01,  1.1645479e-01, -1.2791230e-01,
        2.7641625e-05, -9.3722373e-02,  1.5119699e-01, -9.8118551e-02,
        6.1052620e-02, -3.0996993e-01,  9.3275547e-02, -2.3359668e-02,
        3.0665058e-01,  3.2136410e-02,  3.5929924e-01,  1.1415872e-01],
      dtype=float32)

In [39]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

NameError: name 'enc_state_infer' is not defined

In [ ]:
_enc_state_infer.shape

In [ ]:
_means_topic_infer[0]